In [1]:
from pamona import Pamona
import numpy as np
import pamona

from unioncom import UnionCom

In [12]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [1]:
import os
os.environ['R_HOME'] = '/risapps/rhel7/R/3.6.0/lib64/R' #path to your R installation
os.environ['R_LIBS_USER'] = "/rsrch3/home/bcb/sliang3/R/x86_64-pc-linux-gnu-library/3.6:/risapps/rhel7/R/3.6.0/lib64/R/library"
import rpy2
%load_ext rpy2.ipython

In [ ]:
suffix = '0.5'
file_name = 'Ncell2000_PopSize_nGene500_nPeak1500_w' + suffix + '.RDS'

In [2]:
%%R -o rna -o atac -i file_name
data <- readRDS(file_name)
rna <- data$X
atac <- data$Y

In [3]:
print(rna.dim)
print(atac.dim)

[1]  500 3000

[1] 1000  800



In [6]:
#random_index = np.random.choice(30672, 5000, False)

data1 = np.array(rna).T #[random_index, :]
data2 = np.array(atac).T #[random_index, :]

shuffle = np.arange(data2.shape[0])
np.random.shuffle(shuffle)
shuffle_back = np.argsort(shuffle)

data2 = data2[shuffle, :]

In [7]:
from sklearn.decomposition import PCA

##type1 = LabelEncoder().fit_transform(meta['celltype1'])#[random_index]
##type2 = LabelEncoder().fit_transform(meta['celltype1'])#[random_index]

data1 = pamona.utils.zscore_standardize(np.asarray(data1))
data1 = PCA(30).fit_transform(data1)
data1 = pamona.utils.zscore_standardize(data1)

data2 = pamona.utils.zscore_standardize(np.asarray(data2))
data2 = PCA(30).fit_transform(data2)
data2 = pamona.utils.zscore_standardize(np.asarray(data2))

data = [data1,data2]
#datatype = [type1,type2]

In [9]:
## incorporate prior information
#M = []
#n_datasets = len(data)
#for k in range(n_datasets-1):
#    M.append(np.ones((len(data[k]), len(data[-1]))))
#    for i in range(len(data[k])):
#        for j in range(len(data[-1])):
#            if datatype[k][i] == datatype[-1][j]:
#                M[k][i][j] = 0.5

In [10]:
%%R

rm(data)
gc()

           used  (Mb) gc trigger   (Mb)  max used  (Mb)
Ncells  1371679  73.3    2548180  136.1   2548180 136.1
Vcells 95388134 727.8  155101335 1183.4 106729686 814.3


In [ ]:
uc = UnionCom.UnionCom()
integrated_data = uc.fit_transform(dataset=[data1,data2])

Pamona start!
use random seed: 666


In [ ]:
embedding = pd.DataFrame(integrated_data[0],
                         columns=['X_' + str(i) for i in range(integrated_data[0].shape[1])])
embedding.to_csv("output/sim4-" + suffix + "-unioncom-rna.csv")

embedding = pd.DataFrame(integrated_data[1][shuffle_back, :],
                         columns=['Y_' + str(i) for i in range(integrated_data[1].shape[1])])
embedding.to_csv("output/sim4-" + suffix + "-unioncom-atac.csv")

In [ ]:
uc.Visualize([data1,data2], integrated_data, mode='PCA')